In [1]:
import datetime
import pandas as pd

In [2]:
def na_table(x):
    na = pd.DataFrame(x.isna().sum())
    nna = pd.DataFrame(x.notna().sum())
    na.reset_index(inplace=True)
    nna.reset_index(inplace=True)
    na = na[na[0]>0]
    na = na.merge(nna, on='index')
    na.columns = ['col', 'isNA', 'notNA']
    return na

In [3]:
data_dir = '../train/'
features_dir = '../features/'
change_info = pd.read_csv(data_dir + 'change_info.csv',encoding = 'gbk')
change_info.drop_duplicates(inplace=True)
na_table(change_info)

,col,isNA,notNA


In [4]:
change_info.drop_duplicates(inplace=True)
change_info.bgrq = change_info.bgrq.astype('str')
change_info.bgrq = change_info.bgrq.apply(lambda x:x.split('.')[0])
change_info.bgrq = pd.to_datetime(change_info.bgrq)

In [5]:
base_info = pd.read_csv(data_dir + 'base_info.csv',encoding='gbk')
base_info.drop_duplicates(inplace=True)
base_info.opfrom = pd.to_datetime(base_info.opfrom)
base_info.opfrom = base_info.opfrom.apply(lambda x:datetime.datetime.now() - x)
base_info.opfrom = base_info.opfrom.apply(lambda x:x.days/365)
base_info = pd.DataFrame([base_info.id, base_info.opfrom]).T
base_info.columns = ['id', 'years']
base_info.head()

,id,years
0,47645761dc56bb8c5fae00114b768b5d9b6e917c3aec07c4,1.29589
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,3.13973
2,59b38c56de3836838082cfcb1a298951abfe15e6940c49ba,0.115068
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,5.07671
4,f000950527a6feb63ee1ce82bb22ddd1ab8b8fdffa3b91fb,2.90411


In [6]:
change_info['year'] = change_info.bgrq.apply(lambda x:x.year)
# change_info['month'] = change_info.bgrq.apply(lambda x:x.month)
# change_info['day'] = change_info.bgrq.apply(lambda x:x.day)
change_info.drop(columns=['bgxmdm', 'bgq', 'bgh', 'bgrq'], inplace=True)
change_info.head()

,id,year
0,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2019
1,9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3,2019
2,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,2017
3,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,2017
4,e9f7b28ec10e047000d16ab79e1b5e6da434a1697cce7818,2017


In [7]:
change_info = pd.DataFrame(change_info.id.value_counts())
change_info.reset_index(inplace=True)
change_info.columns = ['id', 'change']
change_info = change_info.merge(base_info, on='id')
change_info['avg_change'] = change_info.change / change_info.years
change_info.drop(columns='years', inplace=True)
change_info.head()

,id,change,avg_change
0,516ab81418ed215d70d2992c79fb2c89f9bfd197e74ff49b,57,13.9257
1,f000950527a6feb6e856468300290e200429d5633a98e6e4,51,2.5064
2,f000950527a6feb68bfd4bea2542528d5c680ab0c989b1e9,51,2.76433
3,f000950527a6feb69ae213e665a842c1e52002192fd39291,51,5.36301
4,f000950527a6feb6a1ea5fcade7c775a1dc94bd7b289d3f0,49,1.92291


In [8]:
change_info.set_index(keys='id', inplace=True)
change_info.to_csv(features_dir + 'change_info.csv')